# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: 37
* Students: Adélie Eliane Garin, Celia Camille Hacker, Isabela Constantin, Michael Spieler
* Dataset: Wikipedia https://snap.stanford.edu/data/wikispeedia/wikispeedia_paths-and-graph.tar.gz

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# for replicating Q8
np.random.seed(seed=37)

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [3]:
data_path = '../wikispeedia_paths-and-graph/'

In [4]:
def decode_utf8(string):
    parts = string.encode('ascii').split(b'%')
    decoded = [bytes.fromhex(part[:2].decode('ascii')) + part[2:] for part in parts[1:]]
    raw = parts[0] + b''.join(decoded)
    return raw.decode('utf-8')

In [5]:
articles = pd.read_csv(data_path+ 'articles.tsv', sep='\t', names=['article'], skiprows=11)
articles.head(10)
articles['article'] = articles['article'].apply(decode_utf8)

In [6]:
articles.head(10)

,article
0,Áedán_mac_Gabráin
1,Åland
2,Édouard_Manet
3,Éire
4,Óengus_I_of_the_Picts
5,€2_commemorative_coins
6,10th_century
7,11th_century
8,12th_century
9,13th_century


In [7]:
categories = pd.read_csv(data_path+ 'categories.tsv', sep='\t', names=['category'], skiprows=12).reset_index()
categories.rename(columns={"index": "article"}, inplace=True)
categories['article'] = categories['article'].apply(decode_utf8) 
categories['category'] = categories['category'].apply(lambda x: [x])
categories.head()

,article,category
0,Áedán_mac_Gabráin,[subject.History.British_History.British_Histo...
1,Áedán_mac_Gabráin,[subject.People.Historical_figures]
2,Åland,[subject.Countries]
3,Åland,[subject.Geography.European_Geography.European...
4,Édouard_Manet,[subject.People.Artists]


In [8]:
article_df = categories.groupby(by= 'article').agg({'category': 'sum'}).reset_index()
article_df.head(10)

,article,category
0,10th_century,[subject.History.General_history]
1,11th_century,[subject.History.General_history]
2,12th_century,[subject.History.General_history]
3,13th_century,[subject.History.General_history]
4,14th_century,[subject.History.General_history]
5,15th_Marine_Expeditionary_Unit,[subject.History.Military_History_and_War]
6,15th_century,[subject.History.General_history]
7,16_Cygni,[subject.Science.Physics.Space_Astronomy]
8,16_Cygni_Bb,[subject.Science.Physics.Space_Astronomy]
9,16th_century,[subject.History.General_history]


In [9]:
#sanity check
article_df.loc[article_df['article'] == 'Áedán_mac_Gabráin']['category'].iloc[0]

['subject.History.British_History.British_History_1500_and_before_including_Roman_Britain',
 'subject.People.Historical_figures']

In [10]:
article_df.shape

(4598, 2)

We used `article_df` instead of `features`

In [11]:
# Your code here.
 
#features = # the pandas dataframe with the features and labels

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [12]:
edges = pd.read_csv(data_path + 'links.tsv', sep='\t', names=['article', 'link'], skiprows=12)
edges['article'] = edges['article'].apply(decode_utf8) 
edges['link'] = edges['link'].apply(decode_utf8) 
edges.head()

,article,link
0,Áedán_mac_Gabráin,Bede
1,Áedán_mac_Gabráin,Columba
2,Áedán_mac_Gabráin,Dál_Riata
3,Áedán_mac_Gabráin,Great_Britain
4,Áedán_mac_Gabráin,Ireland


In [13]:
# Note there are links being linked but not having a category
set(list(edges['link'])) - set(list(article_df['article']))

{'Directdebit', 'Friend_Directdebit', 'Pikachu'}

In [14]:
# Note there are links that link to other articles but dont have a category
set(list(edges['article'])) - set(list(article_df['article']))

{'Friend_Directdebit', 'Pikachu', 'Sponsorship_Directdebit'}

We add these pages to our article_df 

In [15]:
article_df = article_df.merge(edges.drop(columns=['link']).drop_duplicates(), \
                              right_on= 'article', left_on= 'article', how='outer' )
print(len(article_df))

4601


In [16]:
edges_temp = edges.drop(columns=['article']).drop_duplicates().rename(columns = {'link': 'article'})
article_df = article_df.merge(edges_temp, right_on= 'article', left_on= 'article', how='outer')
article_df.head()

,article,category
0,10th_century,[subject.History.General_history]
1,11th_century,[subject.History.General_history]
2,12th_century,[subject.History.General_history]
3,13th_century,[subject.History.General_history]
4,14th_century,[subject.History.General_history]


In [17]:
article_df.shape

(4602, 2)

In [18]:
nodes = article_df.reset_index(level=0)
nodes.drop(columns=['category'], inplace=True)
nodes.rename(columns={'index':'idx'}, inplace=True)
nodes.head()

,idx,article
0,0,10th_century
1,1,11th_century
2,2,12th_century
3,3,13th_century
4,4,14th_century


In [19]:
# map nodes to indicies
node_map = dict(zip( nodes.article, nodes.idx))

In [20]:
edges['article_idx'] = edges['article'].apply(lambda x: node_map[x])
edges['link_idx'] = edges['link'].apply(lambda x: node_map[x])
edges = edges.drop(columns=['article', 'link'])
edges.head()

,article_idx,link_idx
0,4592,524
1,4592,966
2,4592,1260
3,4592,1763
4,4592,2140


In [21]:
n_nodes = len(nodes)
adjacency = np.zeros((n_nodes, n_nodes), dtype=int)

for idx, row in edges.iterrows():
    if np.isnan(row.link_idx):
        continue
    i, j = int(row.article_idx), int(row.link_idx)
    
    # TODO: which index is which? this matters since it is a directed graph 
    # -> node i (row i) links to node j (column j)
    adjacency[i, j] = 1


In [22]:
n_nodes

4602

In [23]:
# sanity checks 
adjacency[1][3002] == 1 # there is a link between page 1 and 3002
edges.loc[(edges['article_idx'] == 1) & (edges['link_idx'] ==  3002)]

,article_idx,link_idx
183,1,3002


## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [24]:
plt.spy(adjacency)
plt.title('adjacency matrix')

Text(0.5,1.05,'adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [25]:
n_nodes = len(nodes)

L = np.sum(adjacency)
# This sums the links in the directed graph: if we have A->B and B->A then it counts as two links. 
# If we wanted this to count as one link we would have to do the same computation on the undirected adjacency matrix and divide by 2 

L_max_undirected = int(n_nodes*(n_nodes-1)/2)
# Again, n_nodes*(n_nodes-1)/2) is the undirected case. In the directed case there can be two links between a node A and a node B. 
L_max = int(n_nodes*(n_nodes-1))
# We multiplied by 2 because the maximal number of links is "doubled" by A->B and B->A (need to count them twice)

print('L = {}, L_max = {}, sparsity = {:.4f}'.format(L, L_max, L/L_max))


L = 119882, L_max = 21173802, sparsity = 0.0057


*Answer* 
Clearly L << L_max as in many real world networks. It makes sense here as many wikipedia pages (like 'cats') will not be linked to other unrelated subjects (like 'spaceships') ;)
We can also see the sparsity in the adjacency matrix above. 

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**Your answer here.**

*Answer:*
Our graph is directed since a URL link on a Wiki page is directed.
To make it undirected the adjacency matrix can be OR-ed with its transposed such that it is symmetric.

In [26]:
adjacency_undirected = np.maximum(adjacency, adjacency.T)
plt.spy(adjacency_undirected)

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [27]:
# Your code here.
edges.to_csv('edges.csv')
article_df.to_csv('article_dv.csv')
#np.save('adjacency_sym.npy', adjacency_sym)
np.savez_compressed('adjacency_undirected.npz', adjacency_undirected)

### NOTE: For the following questions we consider only the undirected graph!

### Question 4

Are the edges of your graph weighted?

**Your answer here.**
No, links on a Wikipedia page are not weighted.

### Question 5

What is the degree distibution of your network? 

In [28]:
degree =  np.sum(adjacency_undirected, axis=0)

assert len(degree) == n_nodes



Execute the cell below to see the histogram of the degree distribution.

In [29]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [30]:
# Considering the undirected graph
L_undirected= np.sum(adjacency_undirected)/2
# We compute the number of links in the undirected case as this will differ from the directed case


print ('The average degree in the network is ' + str(2*L_undirected/n_nodes))

The average degree in the network is 46.32420686657975


### Question 6

Comment on the degree distribution of your network. 



**Your answer here.**

*Answer*
We have 4602 nodes with an average degree of 46.32 (in the undirected network). Compared to other internet networks this is much higher. For example the network consisting of webpages, the average is only 4.6. It is not surprising in the case of Wikipedia as there are many links in every wikipedia page.   



### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [31]:
# a queue data structure for BFS
class Queue:
    def __init__(self):
        self.elem = []

    def isEmpty(self):
        return (len(self.elem) == 0)

    def enqueue(self, item):
        self.elem.append(item)

    def dequeue(self):
        return self.elem.pop(0)

    def size(self):
        return len(self.items)

In [32]:
# pseudocode: start BFS from node 0. if by the end the number of visited nodes < nb of nodes 
# then the graph is disconnected
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    start_node = 0
    node_q = Queue()
    node_q.enqueue(start_node)
    visited = set()
    visited.add(start_node)
    nb_visited = 1
    while (node_q.isEmpty() == False ):
        curr = node_q.dequeue()
        successors = adjacency[curr].nonzero()[0]
        for succ in successors:
            if succ not in visited:
                node_q.enqueue(succ)
                visited.add(succ)
                nb_visited += 1
    connected = (nb_visited == adjacency.shape[0])
    if connected:
        print('The graph is connected')
    else:
        print('The graph is not connected')
        print('The number of visited nodes starting from ', start_node, ' is ', nb_visited)
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [31]:
# Your code here.
connected_graph(adjacency_undirected)

The graph is not connected
The number of visited nodes starting from  0  is  4589


False

### Question 8

Write a function that extracts the connected components of a graph.

In [33]:
# similar approach as in previous question, but add an outer for loop in order to go through all connected 
# of the graph 
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    n_nodes = adjacency.shape[0]
    components = []
    is_visited_global = np.zeros(n_nodes, dtype=bool)
    for node in range(n_nodes):
        if is_visited_global[node] == False:
            start_node = node
            node_q = Queue()
            node_q.enqueue(start_node)
            visited = set()
            visited.add(start_node)
            is_visited_global[start_node]= True
            while (node_q.isEmpty() == False ):
                curr = node_q.dequeue()
                successors = adjacency[curr].nonzero()[0]
                for succ in successors:
                    if succ not in visited:
                        node_q.enqueue(succ)
                        visited.add(succ)
                        is_visited_global[succ] = True
            # now a component has been found, add it to the list of adj matricies
            idx_comp = list(visited)
            components.append(adjacency[idx_comp][:,idx_comp])
    return components

Note:
* we could have used a single function to do BFS over a connected component in both Q 7 and 8 to avoid code repetition, but to go by the required API, we decided to stick to writing two independent functions. 

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [34]:
# Your code here.
connected_comp = find_components(adjacency_undirected)
print('The number of connected components in the graph is ', len(connected_comp))
idx_larg_comp = np.argmax([len(adj) for adj in connected_comp])
adj_larg_comp = connected_comp[idx_larg_comp]
nb_nodes_larg_comp = len(adj_larg_comp)
print('The largest component has ', nb_nodes_larg_comp, ' nodes')

The number of connected components in the graph is  12
The largest component has  4589  nodes


### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [36]:
# Implements the Djikstra algorithm from Wikipedia: https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Algorithm
# TODO: not shure if this is correct. also, it seems that some of the nodes are not reachable
def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    n_nodes = adjacency.shape[0]
    MAX_DIST = np.inf
    unvisited = set(np.arange(n_nodes))
    shortest_path_lengths = np.full(n_nodes, MAX_DIST)

    shortest_path_lengths[source] = 0
    while unvisited:
        unvisited_list = list(unvisited)
        current = unvisited_list[np.argmin(shortest_path_lengths[unvisited_list])]
        adjacency_list = adjacency[current]
        neighbors = set(np.nonzero(adjacency_list)[0])
        for n in neighbors.intersection(unvisited):
            path_len = shortest_path_lengths[current] + 1
            if shortest_path_lengths[n] > path_len:
                shortest_path_lengths[n] = path_len

        unvisited.remove(current)
    
    return shortest_path_lengths

short_node_0 = compute_shortest_path_lengths(adjacency_undirected, 0)
short_node_0

array([ 0.,  1.,  1., ...,  3., inf, inf])

In [43]:
# For example, list of nodes not reachable by 0 
np.where(short_node_0 == np.inf)

(array([ 435,  962, 1227, 2534, 3341, 3631, 3913, 4278, 4322, 4469, 4598,
        4600, 4601]),)

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [38]:
# Your code here.
# take the largest connected comp
# sample randomly approx 1000 nodes and extract the largest comp from the subsampled graph
sample_idx = np.random.choice(nb_nodes_larg_comp, size = 1100, replace=False)
adj_sample = adj_larg_comp[sample_idx][:,sample_idx]
components_sample = find_components(adj_sample)
idx_larg_comp = np.argmax([len(adj) for adj in components_sample])
adj_sample_conn = components_sample[idx_larg_comp]
# compute the longest shortest path for each node.
nb_n_sample = len(adj_sample_conn)
print('The selected component has ', nb_n_sample, ' nodes')
longest_shortest_paths = [np.max(compute_shortest_path_lengths(adj_sample_conn, node)) for node in range(nb_n_sample)]

The selected component has  1072  nodes


In [39]:
print('The diameter of the largest connected comp of the sub-sampled graph is ', np.max(longest_shortest_paths) )

The diameter of the largest connected comp of the sub-sampled graph is  7.0


### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

Note: for answering this question, we used the following theorem:

Let G a simple undirected graph and A its adjacency matrix. 
The $(i,j)$ th entry of $A^k$ counts the number of walks of length $k$ having source and end vertices
$i$ and $j$ respectively. 

In [41]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    n_paths=int(np.linalg.matrix_power(adjacency, length)[source][target])
    
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

### TODO: change output

In [ ]:
print(compute_paths(adjacency_undirected, 0, 10, 1))
print(compute_paths(adjacency_undirected, 0, 10, 2))
print(compute_paths(adjacency_undirected, 0, 10, 3))
print(compute_paths(adjacency_undirected, 23, 67, 2))
print(compute_paths(adjacency_undirected, 15, 93, 4))

0


### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [59]:
adjacency_undirected_power_3=np.linalg.matrix_power(adjacency_undirected,3)

print(int(np.sum(adjacency_undirected_power_3)))

3702162721


### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

# TODO: check Q 13 and 14 

In [43]:
#  we modified the API to account for the matrix multiplication, in order to do it just once. 
def compute_clustering_coefficient(adjacency, node, power_matrix=None):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    degree = np.sum(adjacency, axis = 0)
    L = np.linalg.matrix_power(adjacency,3)[node][node]/2
    k= degree[node]
    
    clustering_coefficient= L/(k*(k-1)/2)
    
    return clustering_coefficient, power_matrix


In [ ]:
coeff, power_mat = compute_clustering_coefficient(adj_larg_comp,0 )
print('The clustering coeff of node 0 is ', coeff)

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [60]:
# Modifiy this once we have the largest connected component: some nodes are apparently completely isolated 
average_clustering_coefficient=0 
for i in range(nb_nodes_larg_comp):
    coeff, _ = compute_clustering_coefficient(adj_larg_comp, i, power_mat)
    average_clustering_coefficient+= coeff
    
average_clustering_coefficient=average_clustering_coefficient/nb_nodes_larg_comp

print('The average clustering coeffcient of our network is '+str(average_clustering_coefficient))

The average clustering coeffcient of our network is nan


/Users/celiahacker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [49]:
degree - 1

array([ 84,  83, 111, ...,  14,   1,   1])

In [50]:
# another possible solution without using the function defined above and without loop
# Modifiy this once we have the largest connected component 
np.sum(np.diag(adj_larg_comp)/(2*degree_larg_conn_comp*(nb_nodes_larg_comp-1)))/nb_nodes_larg_comp

9.165932371461522e-08

### Unit tests

#### Question 8 

In [170]:
# connected graph
adj_undir = np.array([[0, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0]])
find_components(adj_undir)

[array([[0, 1, 1, 0, 1, 0],
        [1, 0, 1, 1, 0, 1],
        [1, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0],
        [1, 0, 1, 1, 0, 1],
        [0, 1, 0, 0, 1, 0]])]

In [171]:
# disconnect 0 
adj_undir_2 = np.array([[0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0]])
find_components(adj_undir_2)

[array([[0]]), array([[0, 1, 1, 0, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [0, 1, 1, 0, 1],
        [1, 0, 0, 1, 0]])]

In [174]:
# disconnect 5
adj_undir_3 = np.array([[0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])
find_components(adj_undir_3)

[array([[0]]), array([[0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 0]]), array([[0]])]

In [176]:
# have all disconnected
adj_undir_4 = np.array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])
find_components(adj_undir_4)

[array([[0]]), array([[0]]), array([[0]])]